In [1]:
import json
data_dir = r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Json\Consolidated_processed_modified\notes.json"
queries = r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Json\Consolidated_query_processed_modified\notes.json"


In [2]:
with open(data_dir, 'r') as f:
    fname_to_notes = json.load(f)


In [3]:
midiEt_to_note = {
    12: "C",
    13: "C#",
    14: "D",
    15: "D#",
    16: "E",
    17: "F",
    18: "F#",
    19: "G",
    20: "G#",
    21: "A",
    22: "A#",
    23: "B"
}


# a function which converts a sequence of midiEt to a sequence of notes
def midiEt_to_note_sequence(midiEt_sequence):
    note_sequence = []
    for midiEt in midiEt_sequence:
        num = midiEt % 12
        note_sequence.append(midiEt_to_note[num + 12])
    return "".join(note_sequence)


In [4]:
from similarity_sankoff import similarity, Note
import miditoolkit as mtk
# queries=[Note("G",2,rest=False)]

# t=total_time/(4*tempo)
# duration=int(end-start/t)

In [5]:
#obj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_clipped\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1_melody.mid")
obj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_clipped_processed\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1_melody.mid")


In [6]:
from Clarinet.preprocessing import getBPM

In [7]:
notes = obj.instruments[0].notes
#print(notes[:5])
end_time = notes[-1].end
print(end_time)
#tempo = getBPM(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_clipped_processed\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1_melody.mid")
tempo = 120
t = end_time/(4*tempo)
t = 32/768
# ans = 99999
# prev_end = 0
# for n in notes:
#     if n.start > prev_end:
#         dur = n.start - prev_end
#         ans = min(ans, dur)
#     ans = min(ans, n.end - n.start)
#     prev_end = n.end
# t= ans
        


40789


In [8]:
import numpy as np
sum= 0
start = []
end = []
for n in notes:
    start.append(n.start)
    end.append(n.end)
diff = np.array(end) - np.array(start)
print(np.mean(diff))
print(np.std(diff))

844.2894736842105
908.9625349205526


In [9]:
rep = []
prev_end = 0
for n in notes:
    if int((n.start-prev_end)/t) != 0:
        if n.start > prev_end:
            rep.append(Note("C",int((n.start-prev_end)/t),rest=True))
    if int((n.end-n.start)/t) != 0:
        rep.append(Note(midiEt_to_note[n.pitch%12 + 12],int((n.end-n.start)/t),rest=False))
    prev_end = n.end


In [10]:
#qobj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_queries\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1_query_melody.mid")
#qobj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_queries_processed\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1_query_melody.mid")
qobj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\Consolidated_query_processed_modified\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_01_Track01_wav_melody.mid")


In [11]:
notes = qobj.instruments[0].notes
print(notes[:5])
end_time = notes[-1].end
tempo = 120
t = end_time/(4*tempo)
t=350

# ans = 99999
# prev_end = 0
# for n in notes:
#     if n.start > prev_end:
#         dur = n.start - prev_end
#         ans = min(ans, dur)
#     ans = min(ans, n.end - n.start)
#     prev_end = n.end
# t = ans


[Note(start=0, end=256, pitch=45, velocity=53), Note(start=512, end=1536, pitch=68, velocity=61), Note(start=1536, end=1707, pitch=52, velocity=46), Note(start=1707, end=1963, pitch=59, velocity=50), Note(start=2048, end=2304, pitch=64, velocity=75)]


In [12]:
from tqdm import tqdm
import os
import music21
fnames = os.listdir(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\Consolidated_processed_modified")
fnames = fnames[:100]

def getKey(midi_filename):
    score = music21.converter.parse(midi_filename)
    key = score.analyze('key')
    return(key.tonic.name)
def getMode(midi_filename):
    score = music21.converter.parse(midi_filename)
    key = score.analyze('key')
    return(key.mode)


In [13]:
qrep = []
prev_end = 0
mode = getMode(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\Consolidated_query_processed_modified\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_01_Track01_wav_melody.mid")
for n in notes:
    if int((n.start-prev_end)/t) != 0:
        if n.start > prev_end:
            qrep.append(Note("C", int((n.start-prev_end)/t), rest=True,scale=mode))
    if int((n.end-n.start)/t) != 0:
        qrep.append(Note(midiEt_to_note[n.pitch % 12 + 12], int((n.end-n.start)/t), rest=False,scale=mode))
    prev_end = n.end


In [14]:
def compute_similarities(qrep):
    fname_to_similarity = {}
    for fname in fnames:
        sim = 0
        obj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\Consolidated_processed_modified\\" + fname)
        notes = obj.instruments[0].notes 
        # ans = 99999
        # prev_end = 0
        
        # for n in notes:
        #     if n.start > prev_end:
        #         dur = n.start - prev_end
        #         ans = min(ans, dur)
        #     ans = min(ans, n.end - n.start)
        #     prev_end = n.end
        # t = ans
        t = 350
        mode = getMode(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\Consolidated_processed_modified\\" + fname)
        

        rep = []
        prev_end = 0
        for n in notes:
            if int((n.start-prev_end)/t) != 0:
                if n.start > prev_end:
                    rep.append(Note("C", int((n.start-prev_end)/t), rest=True,scale=mode))
            if int((n.end-n.start)/t) != 0:
                rep.append(Note(midiEt_to_note[n.pitch % 12 + 12], int((n.end-n.start)/t), rest=False,scale=mode))
            prev_end = n.end
        sim = similarity(qrep, rep)
        fname_to_similarity[fname] = sim
    return fname_to_similarity


In [15]:
fname_to_similarity = compute_similarities(qrep)

22it [00:00, ?it/s]                  
62it [00:00, 1885.89it/s]             
20it [00:00, 2363.72it/s]            
66it [00:00, 6531.18it/s]             
82it [00:00, 4337.17it/s]             
44it [00:00, 5453.43it/s]             
53it [00:00, 6574.92it/s]             
29it [00:00, 3623.43it/s]            
59it [00:00, 3591.95it/s]             
16it [00:00, 1814.83it/s]            
54it [00:00, 3122.10it/s]             
70it [00:00, 3920.49it/s]             
59it [00:00, 3472.64it/s]             
45it [00:00, 2809.77it/s]             
25it [00:00, 3123.36it/s]            
59it [00:00, 6972.19it/s]             
48it [00:00, 2813.95it/s]             
47it [00:00, 5792.73it/s]             
76it [00:00, 4774.68it/s]             
46it [00:00, 5728.73it/s]             
60it [00:00, 3599.03it/s]             
20it [00:00, 2486.91it/s]            
17it [00:00, 1642.97it/s]            
47it [00:00, 5844.25it/s]             
32it [00:00, 3966.36it/s]            
62it [00:00, 7517.11it/s]        

In [16]:
dict(sorted(fname_to_similarity.items(), key=lambda item: item[1], reverse=True))

{'MIDI-Unprocessed_02_R1_2008_01-05_ORIG_MID--AUDIO_02_R1_2008_wav--1_melody.mid': 1.0,
 'MIDI-Unprocessed_02_R1_2008_01-05_ORIG_MID--AUDIO_02_R1_2008_wav--3_melody.mid': 1.0,
 'MIDI-Unprocessed_03_R1_2009_03-08_ORIG_MID--AUDIO_03_R1_2009_03_R1_2009_06_WAV_melody.mid': 0.5,
 'MIDI-Unprocessed_02_R1_2008_01-05_ORIG_MID--AUDIO_02_R1_2008_wav--5_melody.mid': 0.3749999999999999,
 'MIDI-UNPROCESSED_04-05_R1_2014_MID--AUDIO_05_R1_2014_wav--1_melody.mid': 0.25,
 'MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R1_2014_wav--3_melody.mid': 0.22499999999999998,
 'MIDI-Unprocessed_01_R1_2008_01-04_ORIG_MID--AUDIO_01_R1_2008_wav--3_melody.mid': 0.22499999999999998,
 'MIDI-Unprocessed_01_R1_2011_MID--AUDIO_R1-D1_06_Track06_wav_melody.mid': 0.22499999999999998,
 'MIDI-Unprocessed_02_R2_2011_MID--AUDIO_R2-D1_02_Track02_wav_melody.mid': 0.22499999999999998,
 'MIDI-Unprocessed_03_R1_2008_01-04_ORIG_MID--AUDIO_03_R1_2008_wav--1_melody.mid': 0.22499999999999998,
 'MIDI-Unprocessed_03_R1_2009_03-08_ORIG_MID-

In [17]:
fname_to_similarity["MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_01_Track01_wav_melody.mid"]


0.0

In [18]:
similarity([Note("C", 10), Note("D", 30)], [Note("C", 10), Note("D", 50), Note("E", 30)])


100%|██████████| 2/2 [00:00<?, ?it/s]


-19.0